In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

In [16]:
training_data = datasets.FashionMNIST(
    root="../../0. DataSet/2. FashionMNIST/train/",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (1.0))
    ])
)

valid_data = datasets.FashionMNIST(
    root="../../0. DataSet/2. FashionMNIST/valid/",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (1.0))
    ])
)

test_data = datasets.FashionMNIST(
    root="../../0. DataSet/2. FashionMNIST/test/",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (1.0))
    ])
)

In [17]:
batch_size = 1

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# train_dataloader = DataLoader(training_data)
# test_dataloader = DataLoader(test_data)

In [18]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([1, 1, 28, 28])
Shape of y: torch.Size([1]) torch.int64


In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [20]:
model = NN().to(device)

print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
from torch import optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)

            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

In [24]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.293673 [    0/60000]
loss: 2.455520 [  100/60000]
loss: 2.473237 [  200/60000]
loss: 2.292160 [  300/60000]
loss: 2.347889 [  400/60000]
loss: 2.199419 [  500/60000]
loss: 2.237597 [  600/60000]
loss: 2.274113 [  700/60000]
loss: 2.125730 [  800/60000]
loss: 2.241529 [  900/60000]
loss: 2.041757 [ 1000/60000]
loss: 2.194345 [ 1100/60000]
loss: 2.190506 [ 1200/60000]
loss: 2.161264 [ 1300/60000]
loss: 2.173455 [ 1400/60000]
loss: 1.843652 [ 1500/60000]
loss: 2.090208 [ 1600/60000]
loss: 1.890355 [ 1700/60000]
loss: 1.907688 [ 1800/60000]
loss: 1.611150 [ 1900/60000]
loss: 1.835714 [ 2000/60000]
loss: 1.171033 [ 2100/60000]
loss: 1.047226 [ 2200/60000]
loss: 1.733765 [ 2300/60000]
loss: 1.933388 [ 2400/60000]
loss: 1.413302 [ 2500/60000]
loss: 2.315315 [ 2600/60000]
loss: 2.380702 [ 2700/60000]
loss: 1.433575 [ 2800/60000]
loss: 1.726766 [ 2900/60000]
loss: 1.628087 [ 3000/60000]
loss: 1.584489 [ 3100/60000]
loss: 1.210888 [ 3200/60000]
los

In [25]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [26]:
model = NN()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: " {predicted}", Actual: "{actual}"')


Predicted: " Ankle boot", Actual: "Ankle boot"
